<a href="https://colab.research.google.com/github/RiccardoRubini93/SWA_DQ/blob/main/Use_pyspark_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [3]:
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [4]:
!ls -al

total 219216
drwxr-xr-x  1 root root      4096 Feb 27 07:26 .
drwxr-xr-x  1 root root      4096 Feb 27 07:25 ..
drwxr-xr-x  4 root root      4096 Feb 23 14:37 .config
drwxr-xr-x  1 root root      4096 Feb 23 14:38 sample_data
drwxr-xr-x 13 1000 1000      4096 Jun  6  2020 spark-3.0.0-bin-hadoop3.2
-rw-r--r--  1 root root 224453229 Jun  6  2020 spark-3.0.0-bin-hadoop3.2.tgz


In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [6]:
!pip install -q findspark

In [7]:
import findspark
findspark.init()

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [10]:
spark

In [12]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2023-02-27 07:33:57--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.28M  5.90MB/s    in 2.2s    

2023-02-27 07:34:00 (5.90 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://f94d-34-80-77-107.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,

In [9]:
#connect to GCP 
from google.colab import auth
auth.authenticate_user()

In [34]:
#copy file locally 

!gsutil cp gs://swarovski-advanced-analytics-data-ingestion-test/raw/50_adyen/5004__ad_payment/v2/2023/5004__ad_payment_SwarovskiCOMAT__dd_part__2023-01-01T08:43:26+01:00.json .


Copying gs://swarovski-advanced-analytics-data-ingestion-test/raw/50_adyen/5004__ad_payment/v2/2023/5004__ad_payment_SwarovskiCOMAT__dd_part__2023-01-01T08:43:26+01:00.json...
- [1 files][  379.0 B/  379.0 B]                                                
Operation completed over 1 objects/379.0 B.                                      


In [38]:
!mkdir tmp

In [42]:
!mv /tmp/5004__ad_payment_SwarovskiCOMAT__dd_part__2023-01-01T08:43:26+01:00.json /tmp/test.json 

In [43]:
#load json in spark df

df = spark.read.json("/tmp/test.json")

In [44]:
df.printSchema()

root
 |-- acquirerresponse: string (nullable = true)
 |-- alias: string (nullable = true)
 |-- amountcurrency: string (nullable = true)
 |-- amountvalue: long (nullable = true)
 |-- cardbin: string (nullable = true)
 |-- cardsummary: string (nullable = true)
 |-- expirymonth: string (nullable = true)
 |-- expiryyear: string (nullable = true)
 |-- issuercountry: string (nullable = true)
 |-- merchantaccountcode: string (nullable = true)
 |-- merchantreference: string (nullable = true)
 |-- pspreference: string (nullable = true)
 |-- shoppercountry: string (nullable = true)
 |-- shopperinteraction: string (nullable = true)
 |-- store: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [45]:
df.show()

+----------------+-----+--------------+-----------+-------+-----------+-----------+----------+-------------+-------------------+------------------+----------------+--------------+------------------+-----+-------------------+
|acquirerresponse|alias|amountcurrency|amountvalue|cardbin|cardsummary|expirymonth|expiryyear|issuercountry|merchantaccountcode| merchantreference|    pspreference|shoppercountry|shopperinteraction|store|          timestamp|
+----------------+-----+--------------+-----------+-------+-----------+-----------+----------+-------------+-------------------+------------------+----------------+--------------+------------------+-----+-------------------+
|                |     |           EUR|       6395|       |           |           |          |           AT|     SwarovskiCOMAT|10AT00000521396P01|B82N6KVF8K5SGHG2|              |         Ecommerce|     |2023-01-01 07:43:26|
+----------------+-----+--------------+-----------+-------+-----------+-----------+----------+------